# Imports

In [2]:
%load_ext autoreload
%autoreload 2

In [31]:
import matplotlib.pyplot as plt
import pandas as pd

import networkx as nx
from pyvis.network import Network

from IPython.core.display import display, HTML

from sklearn.preprocessing import MinMaxScaler

import base64
from io import BytesIO

C:\Users\fricl\AppData\Local\Temp\ipykernel_17568\634483913.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [4]:
#!pip install pyvis
#!pip install jinja2

[Tutorial](https://pyvis.readthedocs.io/en/latest/tutorial.html#getting-started)

In [50]:
data = pd.read_csv("data/country_clicks_links.csv", index_col=0)
articles = data.index.tolist()
clicks = [f"{data.click_count.iloc[i]} clicks" for i in range(len(articles))]

country_clicks = data.groupby('Top_1_name')['click_count'].sum().reset_index()
top_1_counts = data['Top_1_name'].value_counts()
country_clicks['occurrences'] = country_clicks['Top_1_name'].map(top_1_counts)
country_clicks["scaled_click_count"] = country_clicks["click_count"] / country_clicks["occurrences"]

countries = country_clicks.Top_1_name.tolist()
clicks = country_clicks.scaled_click_count.tolist()
#clicks = country_clicks.click_count.tolist()
clicks_label = [f"{clicks[i]} clicks" for i in range(len(clicks))]
num_articles = country_clicks.occurrences.tolist()

In [43]:
country_clicks.sort_values(by='scaled_click_count', ascending=False)

,Top_1_name,click_count,occurrences,scaled_click_count
6,antarctica,8066,24,336.083333
73,gibraltar,1753,6,292.166667
102,kuwait,778,3,259.333333
160,russia,13688,83,164.915663
100,kenya,1803,11,163.909091
...,...,...,...,...
120,martinique,26,2,13.000000
9,aruba,26,2,13.000000
27,british indian ocean territory,12,1,12.000000
190,togo,31,3,10.333333


Let's construct a graph in which: 
- each node is a country 
- the size of a node is proportional to the click count 
- the color of a node is proportional to the number of articles on a grey-to-red scale

In [52]:
# color of nodes is proportional to number of articles associated with the node (country)
scaler = MinMaxScaler()
normalized_counts = scaler.fit_transform([[count] for count in num_articles]).flatten()

color_map = plt.cm.get_cmap('Reds')
colors_hex = [matplotlib.colors.to_hex(color_map(norm)) for norm in normalized_counts]

# size of nodes is proportional to the click count 
size_scaler = MinMaxScaler(feature_range=(9, 337))
node_sizes = size_scaler.fit_transform([[count] for count in clicks]).flatten()


net = Network(directed=True, 
              notebook=True, 
              font_color='#10000000')

net.add_nodes(countries, 
              title=countries, 
              color=colors_hex, 
              size=node_sizes)

"""
# Let's add edges between articles that are connected in Wikipedia
for i, article1 in enumerate(articles[:50]):
    print(article1)
    name_links_out = country_clicks.name_links_out.iloc[i]
    print(name_links_out)

    if pd.notna(name_links_out):
        for article2 in articles[:50]: 
            if article2 in name_links_out:
                print(article2, "is an out link from", article1)
                net.add_edge(article1, article2)
        print("---------")
"""

net.show("graph_scaled.html")

graph_scaled.html


C:\Users\fricl\AppData\Local\Temp\ipykernel_17568\3119077137.py:5: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  color_map = plt.cm.get_cmap('Reds')
